# 변화 감지기

주기적으로 웹사이트에서 무엇이 변했는지 감지한다. 

In [64]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError

import bs4
from bs4 import BeautifulSoup as bs

In [72]:
import difflib
from pprint import pprint
import re

In [3]:
target_sites = ['http://www.coolenjoy.net/bbs/40', 'https://ddd.com'] # DC예수게이 갤러리 안된다. 

In [4]:
def error_checker(site):
    try:
        html = urlopen(site)
    except HTTPError as e:
        print(site, "The server returned an HTTP error, ", e)
        return 1
    except URLError as e:
        print(site, "The server could not be found. ", e)
        return 1
    else: # Success
        return 0

In [5]:
filtered_target_sites = list(map(lambda x: x if error_checker(x)==0 else None, target_sites))
filtered_target_sites = list(filter(None, filtered_target_sites))
filtered_target_sites

https://ddd.com The server could not be found.  <urlopen error [Errno 11001] getaddrinfo failed>


['http://www.coolenjoy.net/bbs/40']

## 사이트의 변화 전후 저장

사용되는 사이트: 거의 죽어있는 사이트, 쿨앤조이 VR 정보포럼 http://www.coolenjoy.net/bbs/40

In [6]:
site = 'http://www.coolenjoy.net/bbs/40'

In [7]:
html_before = urlopen(site)
html_bs_before = bs(html_before, 'html.parser')

In [8]:
html = urlopen(site)
html_bs = bs(html, 'html.parser')

In [70]:
html_bs.descendents == None # 왜 descendents는 비어있는걸까? 

True

In [55]:
soup = bs('<b class="boldest">Extremely bold</b>')
tag = soup.b
type(tag)

bs4.element.Tag

In [49]:
pprint(html_bs.body.

None


In [67]:
aa = []

for i in html_bs:
    aa.append(i)

for i in range(0, len(aa)):
    print(type(aa[i]))

<class 'bs4.element.Doctype'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'

In [69]:
for i in aa:
    if isinstance(i, bs4.element.Tag):
        pprint(i)
        print("----"*10)

<html lang="ko">
<head>
<meta content="59l5b644q1s31r7ahwnmtgclm6t91qv28fpxyx0w-5ps0sn1oa-bdt46f7h2wyf99rno8ond7sdht-3z9wphjsppkr5g5oztcw8wwiai9upu6jg4nh-dyzhtbqcraf8x" name="norton-safeweb-site-verification"/>
<meta charset="utf-8"/>
<meta content="쿨엔조이" property="title"/>
<meta content="수냉식쿨러,수랭식쿨러,CPU쿨러,CPU오버클럭,VGA쿨러,오버클럭,PC튜닝,케이스튜닝,케이스,cpu,vga" property="description"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="no" http-equiv="imagetoolbar"/>
<meta content="IE=10,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="NOINDEX, NOFOLLOW" name="ROBOTS"/>
<title>VR 1 페이지 | 쿨엔조이</title>
<link href="http://www.coolenjoy.net/skin/popular/basic/style.css" rel="stylesheet"/>
<link href="http://www.coolenjoy.net/theme/basic/skin/outlogin/basic/style.css?ver=161020" rel="stylesheet"/>
<link href="http://www.coolenjoy.net/skin/board/basic/style.css?ver=161020" rel="stylesheet"/>
<link href="http://www.coolenjoy.net/theme/basic/css/default.css?ver=161020" rel="styles

In [21]:
pprint(html_bs.body.contents)

['\n',
 ' 팝업레이어 시작 { ',
 '\n',
 <div id="hd_pop">
<h2>팝업레이어 알림</h2>
<span class="sound_only">팝업레이어 알림이 없습니다.</span></div>,
 '\n',
 <script>
$(function() {
    $(".hd_pops_reject").click(function() {
        var id = $(this).attr('class').split(' ');
        var ck_name = id[1];
        var exp_time = parseInt(id[2]);
        $("#"+id[1]).css("display", "none");
        set_cookie(ck_name, 1, exp_time, g5_cookie_domain);
    });
    $('.hd_pops_close').click(function() {
        var idb = $(this).attr('class').split(' ');
        $('#'+idb[1]).css('display','none');
    });
    $("#hd").css("z-index", 1000);
});
</script>,
 '\n',
 ' } 팝업레이어 끝 ',
 '\n',
 <script src="http://www.coolenjoy.net/plugin/hotkey/jquery.hotkeys.js"></script>,
 '\n',
 <script src="http://www.coolenjoy.net/plugin/hotkey/grz_hot_use.js"></script 
  
<div id="jp-wrap">
 <div id="jb-header">

 <div id="logo_top">
 <div id="logo_top2">
<a href="http://www.coolenjoy.net"><img src="http://www.cooln.net/image/cooln01.gif

## 사이트의 한 게시글을 특정하기

In [78]:
cooln = html_bs.find_all(text=re.compile('오큘'))
cooln

['\r\n                    오큘 리프트s 블랙아웃 현상 때문에 고통받고 있네요                    ',
 '\r\n                    오큘러스 고를 받고 살짝 후회가 듭니다                    ',
 '\r\n                    오큘러스 리프트 S 구매해습니다.                    ']

In [80]:
ocul = cooln[0]
type(cooln[0])

bs4.element.NavigableString

In [93]:
ocul.parent.parent.parent.parent.parent.parent.parent.parent.parent.parent.parent # <!DOCTYPE HTML> 까지 가면 bs object itself가 되고, 그보다 parent는 None이다. 

In [95]:
for parent in ocul.parents:
    if parent is None:
        print(parent)
    else:
        print(parent.name)

a
td
tr
tbody
table
div
form
div
div
[document]


In [100]:
ocul.parent.parent.parent.parent.find_all(text=re.compile('오큘'))

['\r\n                    오큘 리프트s 블랙아웃 현상 때문에 고통받고 있네요                    ',
 '\r\n                    오큘러스 고를 받고 살짝 후회가 듭니다                    ',
 '\r\n                    오큘러스 리프트 S 구매해습니다.                    ']

## 한 게시글을 알았을 때, sibling을 구하기

In [101]:
def find_common_ancestor(child):
    for parent in child.parents:
        if len(parent.find_all(text=re.compile('오큘'))) == 1:
            continue
        if len(parent.find_all(text=re.compile('오큘'))) > 1:
            return parent
        else:
            print('Something went wrong.')
            return

In [103]:
find_common_ancestor(ocul).name

'tbody'

In [104]:
ocul.parent.parent.parent.parent.name

'tbody'